In [69]:
import pathlib

In [ ]:
data_root = 'animal-10/train'
data_root = pathlib.Path(data_root)
classes = []
for item in data_root.iterdir():
    print(str(item).split("\\")[-1])
    classes.append((str(item).split("\\")[-1]))
classes = tuple(classes)

# all_image_paths = list(data_root.glob('*/*'))
# all_image_paths = [str(path) for path in all_image_paths]
# #random.shuffle(all_image_paths)

# image_count = len(all_image_paths)
# label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
# label_to_index = dict((name, index) for index,name in enumerate(label_names))
# all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]

butterfly
cat
chicken
cow
dog
elephant
horse
sheep
spider
squirrel


In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

#from models import *
#from utils import progress_bar

In [ ]:


device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.Resize((32,32)),
#     transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

==> Preparing data..


In [ ]:
trainset = torchvision.datasets.ImageFolder(root='./animal-10/train', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testset = torchvision.datasets.ImageFolder(root='./animal-10/val', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

In [ ]:
trainset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imgshow(img):
    img = img/2 +0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.show()


In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

imgshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


from progressbar import *
progress_bar = ProgressBar()

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(16*5*5,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,10)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,3)
        self.conv2 = nn.Conv2d(6,9,3)
        self.conv3 = nn.Conv2d(9,12,3)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(12*3*3,32)
        self.fc2 = nn.Linear(32,16)
        self.fc3 = nn.Linear(16,10)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, x..size(0))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [68]:
import torch.optim as optim
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
mini_batch = 150

for epoch in range(10):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        #print(len(inputs))
        outputs = net(inputs)
        #print(len(outputs), labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if(i%mini_batch == (mini_batch-1)):
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/mini_batch))
            running_loss = 0.0
print('Finish Training')

C:\Users\jeffp\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   150] loss: 2.302
[1,   300] loss: 2.303
[1,   450] loss: 2.302
[1,   600] loss: 2.303
[1,   750] loss: 2.303
[1,   900] loss: 2.303
[1,  1050] loss: 2.303
[1,  1200] loss: 2.304
[1,  1350] loss: 2.303
[1,  1500] loss: 2.303
[1,  1650] loss: 2.302
[1,  1800] loss: 2.303
[1,  1950] loss: 2.302
[1,  2100] loss: 2.303
[1,  2250] loss: 2.303
[1,  2400] loss: 2.303
[2,   150] loss: 2.303
[2,   300] loss: 2.303
[2,   450] loss: 2.302
[2,   600] loss: 2.302
[2,   750] loss: 2.303
[2,   900] loss: 2.303
[2,  1050] loss: 2.304
[2,  1200] loss: 2.302
[2,  1350] loss: 2.302
[2,  1500] loss: 2.302
[2,  1650] loss: 2.302
[2,  1800] loss: 2.303
[2,  1950] loss: 2.304
[2,  2100] loss: 2.302


KeyboardInterrupt: 

In [ ]:
for epoch in range(20):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if(i%mini_batch == (mini_batch-1)):
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/mini_batch))
            running_loss = 0.0
print('Finish Training')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
print('Acc : %d %%' % (100 * correct / total))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
print('Acc : %d %%' % (100 * correct / total))

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

imgshow(torchvision.utils.make_grid(images))
print('Groundtruth: ',' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
testloader

In [ ]:
trainloader